### Use the BookOps WorldCat wrapper to check for holdings based on a designated library (or list of libraries) and a list of OCLC numbers

##### Import libraries
This section loads the packages needed to work with data, send API requests, and use the BookOps wrapper with OCLC's WorldCat API.

In [ ]:
import pandas as pd
import requests
from bookops_worldcat import WorldcatAccessToken
import time

##### Configure access token
This section contains the authentication details required by the WorldCat API. 'mykey' and 'mysecret' should be updated based on the user's credentials.

In [ ]:
#Configure access token
WORLDCAT_KEY = 'mykey'
WORLDCAT_SECRET = 'mysecret'
SCOPES = 'WorldCatMetadataAPI'

##### Configure files
This section contains the filepath and name of the file that will be read (INPUT_FILE) and the file where results will be saved (OUTPUT_FILE).

In [ ]:
#Configure files
INPUT_FILE = 'FILENAME'
OUTPUT_FILE = 'FILENAME.xlsx'

##### Designate libraries
Enter the OCLC symbol for each library that needs to be checked. Each symbol should be contained in single quotation marks and multiple symbols should be separated by a comma. Ex: ['ZCU', 'PCU', 'HUL', 'NYP', 'YUS']

In [ ]:
#Configure libraries
LIBRARY_SYMBOLS = ['ZCU']

##### Generate an access token
The **get_token** function uses the API credentials specified above to create a token to access the WorldCat API. Tokens expire after twenty minutes, and should automatically refresh within the script.

In [ ]:
#Generate an access token
def get_token():
    return WorldcatAccessToken(
        key=WORLDCAT_KEY,
        secret=WORLDCAT_SECRET,
        scopes=SCOPES
    )

##### Get library holdings data

The **get_holdings_data** function takes an OCLC number from INPUT_FILE and the library designated in LIBRARY_SYMBOLS, queries the WorldCat API, checks whether the library has the bibliographic record in its holdings, and returns the library symbol as LIBRARY and the count of holdings as LIBRARY_HOLDINGS_COUNT. If multiple libraries have been designated in LIBRARY_SYMBOLS, each library with a holding will be included in the resulting LIBRARY field. If no holdings are found, the LIBRARY field will show []. 

In [ ]:
#Get library holdings data
def get_holdings_data(oclc_number, token, library_symbol):
    if isinstance(library_symbol, str):
        library_symbol = [library_symbol]

    try:
        url = f'https://americas.discovery.api.oclc.org/worldcat/search/v2/bibs-holdings'
        headers = {
            'Authorization': f'Bearer {token.token_str}',
            'Accept': 'application/json'
        }

        params = {
            'oclcNumber': str(oclc_number).strip(),
            'heldBySymbol': ','.join(library_symbol),
            'limit': 50
        }
        response = requests.get(url, headers=headers, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        rows = []

        number_of_records = data.get("numberOfRecords", 0)
        if number_of_records > 0:
            first_bib = data['briefRecords'][0]
            if 'institutionHolding' in first_bib and 'briefHoldings' in first_bib['institutionHolding']:
                brief_holdings = first_bib['institutionHolding']['briefHoldings']
                library_symbol = [entry['oclcSymbol'] for entry in brief_holdings if 'oclcSymbol' in entry]
                rows.append({
                    "OCLC_NUMBER": oclc_number,
                    "LIBRARY": library_symbol,
                    "LIBRARY_HOLDINGS_COUNT": len(library_symbol)
                })
            else:
                rows.append({
                    "OCLC_NUMBER": oclc_number,
                    "LIBRARY": [],
                    "LIBRARY_HOLDINGS_COUNT": 0
                })
        else:
            rows.append({
                "OCLC_NUMBER": oclc_number,
                "LIBRARY": [],
                "LIBRARY_HOLDINGS_COUNT": 0
            })
        return rows

    except Exception as e:
        print(f"[ERROR] {oclc_number}: {e}")
        return [{
            "OCLC_NUMBER": oclc_number,
            "LIBRARY": [],
            "LIBRARY_HOLDINGS_COUNT": 0
        }]


##### Get summary holdings data

The **get_summary_data** function takes an OCLC number from INPUT_FILE, queries the WorldCat API, and returns a count of libraries worldwide that hold the item in their collection. This count is a count of the libraries holding the item, not a count of the items held.

In [ ]:
#Get summary data of total holdings in OCLC
def get_summary_data(oclc_number, token):
    try:
        url = f'https://americas.discovery.api.oclc.org/worldcat/search/v2/bibs-summary-holdings'
        headers = {
            'Authorization': f'Bearer {token.token_str}',
            'Accept': 'application/json'
        }
        params = {'oclcNumber': str(oclc_number).strip()}
        response = requests.get(url, headers=headers, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        rows = []

        number_of_records = data.get("numberOfRecords", 0)
        if number_of_records > 0:
            first_bib_summary = data['briefRecords'][0]
            total_holding_count = first_bib_summary['institutionHolding']['totalHoldingCount']
            rows.append({
                "OCLC_NUMBER": str(oclc_number).strip(),
                "TOTAL_LIBRARIES_HOLDING_ITEM": total_holding_count
            })
        else:
            rows.append({
                "OCLC_NUMBER": str(oclc_number).strip(),
                "TOTAL_LIBRARIES_HOLDING_ITEM": 0
            })
        return rows

    except Exception as e:
        print(f"[ERROR] {oclc_number}: {e}")
        return [{
            "OCLC_NUMBER": "None",
            "TOTAL_LIBRARIES_HOLDING_ITEM": 0
        }]


##### Run the workflow

The **main** function performs the following steps:
1. Reads INPUT_FILE
2. Creates an API token
3. Runs through each OCLC number and sends a query to get holdings data based on LIBRARY_SYMBOLS
4. If get_summary = True, runs through each OCLC number and sends a query to get total holdings data across all libraries in OCLC
4. Collects the results of the queries and merges them back with the original data
5. Exports the complete dataset as an Excel file
6. Filters the results to only include items held by any of the libraries included on LIBRARY_SYMBOLS
7. Exports the filtered dataset as an Excel file, or prints a message saying no matches found

Depending on the structure of INPUT_FILE, names of fields may need to be updated. For example, the file structure here uses "OCLC_NUMBER" as a field name. A different file may use "network_number" instead, which means either the script below needs to be updated to use "network_number", or INPUT_FILE needs to be updated to use "OCLC_NUMBER".

In [ ]:
#Run query and export results
def main():
    oclclist_df = pd.read_excel(INPUT_FILE, dtype={'RECORD_ID': str, 'OCLC_NUMBER': str})
    all_results = []

    token = get_token()

    #Set as True to fetch summary holdings data and harvest TOTAL_LIBRARIES_HOLDING_ITEM, set as False if not needed
    get_summary = True

    for _, row in oclclist_df.iterrows():
        oclc_number = row['OCLC_NUMBER']
        record_id = row['RECORD_ID']
        if not oclc_number:
            continue

        if token.is_expired():
            print("Refreshing token!")
            token = get_token()

        holdings_rows = get_holdings_data(oclc_number, token, LIBRARY_SYMBOLS)

        if get_summary:
            summary_rows = get_summary_data(oclc_number, token)
            for summary_row in summary_rows:
                for holdings_row in holdings_rows:
                    holdings_row.update(summary_row)

        all_results.extend(holdings_rows)
        print(f"Processed OCLC {oclc_number}")
        time.sleep(0.2)

    holdings_df = pd.DataFrame(all_results)
    merged_df = pd.merge(oclclist_df, holdings_df, on="OCLC_NUMBER", how="left")
    merged_df.to_excel(OUTPUT_FILE, index=False)
    print(f"Data exported to {OUTPUT_FILE}.")

    #Optional filter to export separate file with only rows matching parameters, update or comment out as needed
    library_symbol_str = '_'.join(LIBRARY_SYMBOLS)
    filtered_df = merged_df[merged_df['LIBRARY'].apply(lambda x: any(symbol in str(x) for symbol in LIBRARY_SYMBOLS))]
    if not filtered_df.empty:
        filtered_file = OUTPUT_FILE.replace(".xlsx", f"_Filtered.xlsx")
        filtered_df.to_excel(filtered_file, index=False)
        print(f"Library holdings data exported to {filtered_file}")
    else:
        print(f"No rows matching {LIBRARY_SYMBOLS} found.")

if __name__ == "__main__":
    main()
    